# Fokker-Planck Equation III: Operator Splitting

This notebook investigates operator splitting methods for the Fokker Planck equation. Thus far in the notebook series, we have investigated the two terms in the FPE (Advection and Diffusion) separately, and in this notebook, we will investigate how these independent term-wise solutions can be used in combination to integrate the entire advection-diffusion equation, and discuss why it works, as well as the motivations and runtime savings afforded to the full implementation as a result.

We can write the 1D Fokker-Planck equation, in general, as 

$$ \partial_t p(x, t) = \partial_x[\mu(x, t) p(x, t)] + D \partial_{xx}^2 p(x, t) $$

Which, can be written more concisely in terms of the differential operator $\mathcal{L}$ (often referred to as the *generator*) as

$$ \partial_t p(x, t) = \mathcal{L} \cdot p(x, t) $$

where the Fokker-Planck operator $\mathcal{L}$ is defined as

$$ \mathcal{L} \equiv \left( \partial_x \mu(x, t) + D\partial_{xx}^2 \right) $$

Formally, the solution for $p(x, t)$ of the operator equation is simple (given an initial distribution $p(x, t_0)$), and simply written as

$$ p(x, t) = e^{\mathcal{L} \Delta t} p(x, t_0) $$

however, in practice, this solution is defined only in terms of its Taylor series, and is virtually impossible to evaluate in practice. However, if you consider the limit where only a small amount of time has passed ($\Delta t \ll 1$) then the exponential can be well approximted with its linear-order Taylor series as

$$ p(x, t) \approx p(x, t_0) + \Delta t \mathcal{L}\cdot p(x, t_0) $$

which formally aprpoximates the actual FPE in the $\Delta t \to 0$ limit, as we can re-arrange this equation to read

$$ \frac{p(x, t) - p(x, t_0)}{\Delta t} \approx \partial_t p(x, t) = \mathcal{L}p(x, t_0)$$

So, what does this get us? Well, first, for finite but small timesteps, this equation gives an approximate means of iteratively updating the initial probability (essentialy what we are doing with the advection and diffusion terms in isolation). However, there is a set of methods (bradly known as *operator splitting* methods) that can significantly simplify the calculation of this in practice.

Before delving into how operator splitting works in practice, its worth brining attantion to our prior analysis of the integration schemes for advection and diffusion alone. For the advective equation, the Lax-Wendroff method was sufficient to give high levels of stability, but gets complicated and cumbersome to apply to higher order equations like those involving diffusion. Conversely, a primary consideration for improving wall clock times in the diffusion equation was the fact that we were able to (often) make the simplification to the diffusion update that the coefficient $D$ was constant, and thus we were only required to invert the Crank-Nicolson matrices onces, and then use that inverse going forward (which provided a significant speedup---that improves with increasing $N$---upon inverting a matrix in each iteration). For many dynamic problems (virtually all instances of the Soluchowski equation that we will be interested in) the dynamics enter through the advective term, then it would be nice to still retain the ability to use the constant-$D$ speedup.  Herein lies the benefit of operator splitting methods.  Note that we can generally write the full Fokker-Planck equation as a summation over the advective $\mathcal{L}_{\rm A}$ and diffusive $\mathcal{L}_{\rm D}$ terms, and so the formal solution can be written as

$$ p(x, t) = e^{\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D}}p(x, t_0) $$

Now, if the operators were themselves represented as simple numbers, then we would be able to split up the action of the operator on the initial probability into two contributions explicitlty and treat the operation of each term independently of the others. However, this is (unfortunately) not the case, and so such a procedure will not be correct.

More specifically, we can make an appeal to an important result in the field of Lie algebras---known as the Baker-Campbell-Hausdorff (BCH) formula---to better understand what is going on. For our purposes, the BCH formla states that, for two operators $X$ and $Y$, the followign identity holds

$$ e^Xe^Y = e^Z $$

where

$$ Z \equiv X + Y + \frac{1}{2}[X, Y] - \frac{1}{12}[X, [X, Y]] - \frac{1}{12}[Y, [X, Y]] \cdots $$

where $[X, Y] \equiv XY - YX$ is the commutator beween $X$ and $Y$.

Now, in the context of the present problem, the BCH formula implies that the operator equation solving the Fokker Planck equation can be written as

$$ e^{(\mathcal{L}_{rm A} + \mathcal{L}_{\rm D})\Delta t + \frac{1}{2}[\mathcal{L}_{\rm A}, \mathcal{L}_{\rm D}]\Delta t^2 + \cdots} = e^{\mathcal{L}_{\rm A}\Delta t}e^{\mathcal{L}_{\rm D}\Delta t} $$

which implies that, to linear order in $\Delta t$, the simple decomposition of the composite operator $\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D}$ into the product of individual contributions can accurately capture the same properties.  Formally, this process, which is known as *Lie splitting* breaks the update of the PDE into substeps. In practice, this approximation reduces the update of the composite equation into two sub-problems as

$$ p(x, t + \Delta t) \approx e^{\mathcal{L}_{\rm A}\Delta t} e^{\mathcal{L}_{\rm D}\Delta t}p(x, t) $$

is calculated by first applying the Diffusion update ($\exp(\mathcal{L}_{\rm D}\Delta t)$) and then the advection update ($\exp(\mathcal{L}_{\rm A}\Delta t)$) to the result. From a practical standpoint, this decomposition of integration updates into constituent pieces can be exceptionally convenient as it allows us to independently treat the advection and diffusion parts of the equation, potentially using different integration schemes for each.

While thie approach is convenient as-is, there are still some issues. Namely, there is an ambiguity as to which term to apply first. Given that the commutator between the advection and diffusion operators is assumed to be nonzero, the results will not be the same if the advection or diffusion operators are applied first (although, they will be up to $\Delta t$). Further, the Advection and Diffusion integration schemes we have build for each component are accurate to $\Delta t^2$, so using the Lie splitting scheme eliminates that accuracy, reducing it to $\Delta t$.

This problem can be resolved, however, by instead considering the following (seemingly trivial) modification of the original update operator:

$$ e^{(\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D})\Delta t} = e^{\tfrac{1}{2}(\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D})\Delta t}e^{\tfrac{1}{2}(\mathcal{L}_{\rm D} + \mathcal{L}_{\rm A})\Delta t} $$

which is exact, because an operator will commute with itself, by definition. Note that in the second term on the RHS, the order of the updates has been switched. Now, performing a Lie splitting on both terms from the RHS above gives

$$ e^{\tfrac{1}{2}(\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D})\Delta t + \frac{1}{2}[\mathcal{L}_{\rm A}, \mathcal{L}_{\rm D}]\Delta t^2} = e^{\frac{1}{2}\mathcal{L}_{\rm A}\Delta t}e^{\frac{1}{2}\mathcal{L}_{\rm D}\Delta t} $$
$$ e^{\tfrac{1}{2}(\mathcal{L}_{\rm D} + \mathcal{L}_{\rm A})\Delta t - \frac{1}{2}[\mathcal{L}_{\rm A}, \mathcal{L}_{\rm D}]\Delta t^2} = e^{\frac{1}{2}\mathcal{L}_{\rm D}\Delta t}e^{\frac{1}{2}\mathcal{L}_{\rm A}\Delta t} $$

where in the second operator we have used the identity $[A, B] = -[B, A]$.  Here, it can be seen that, when we combine the two above updates, the $\mathcal{O}(\Delta t^2)$ error terms cancel out, leading to an update scheme that (much like our individual Advection and Diffusion schemes) is accurate to $\mathcal{O}(\Delta t^2)$. Here, the higher order $\Delta t^3$ errors enter through the commultation ov each individual operator with ts commutator (*i.e.* $[\mathcal{L}_{\rm A}, [\mathcal{L}_{\rm A}, \mathcal{L}_{\rm D}]]$). 

Putting this all together the resulting operator decomposition, which in this case is known as *Symmetric Strang Splitting*, is 

$$ p(x, t + \Delta t) \approx e^{-\frac{1}{2}\mathcal{L}_{\rm A}\Delta t}e^{\mathcal{L}_{\rm D}\Delta t} e^{\frac{1}{2}\mathcal{L}_{\rm A}\Delta t}p(x, t) $$

which corresponds to the sequential operations of a 1/2-timestep update of the probability with the advection dynamics, a full time-step update of the diffusive dynamics, followed by another half-step update of the advection dynamics.

Note here that there is still a slight asymmetry in this update equation. The order in which we apply the diffusion and advection updated is not equal. There is another scheme that seeks to remedy this, known as *Symmetrically Weighted Strang Splitting* (SWSS), which is implemented as an option in the integrator, and explained in more detail in Ref[1]

#### References

* [1] W. Press, S. Teukolsky, W. Vetterling, & B. Flannery, *Numerical Recipes: The Art of Scientific Computing*, 3rd Ed., 2007, Cambridge University Press.

In this notebook, we will shows that these splitting schemes do, in fact, succeed in reproducing the correct dynamics of the composite system. Given that we have thusfar developed integration schemes targetted for each term individually, rather than compare the split dynamics to a full integration of the composition (advection + diffusion), we will simply show that the integration split solutions reproduce the same physically relevant features of the dynamics that we would use to judge the efficacy of a full composite integrator.

In [1]:
import os
import numpy as np
import matplotlib as plt
import seaborn as sns

from FPE.Integrator import FPE_Integrator_1D
